In [6]:
import mysql.connector
import pandas as pd
from bs4 import BeautifulSoup
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics import precision_score, recall_score, f1_score
import numpy as np

In [7]:
# Step 1: Connect to MySQL with Error Handling
try:
    connection = mysql.connector.connect(
        host="localhost",
        user="root",
        password="root",
        database="AU_TECH_JOBS"
    )
    cursor = connection.cursor()
    print("Connected to MySQL successfully")
except mysql.connector.Error as err:
    print(f"Error: {err}")
    exit(1)  # Exit the script if the connection fails

Connected to MySQL successfully


In [8]:

# Step 2: Get all table names in the database
cursor.execute("SHOW TABLES")
tables = cursor.fetchall()

In [9]:
# Extract table names
table_names = [table[0] for table in tables]

# Dictionary to hold DataFrames for each table
dataframes = {}

In [10]:
# Step 4: Extract required fields from relevant tables
required_fields = ['description', 'summary', 'position', 'department', 'job_type']
filtered_dataframes = []

In [11]:
try:
    cursor.execute("SELECT * FROM filtered_jobs")
    data = cursor.fetchall()
    columns = [col[0] for col in cursor.description]
    print(f"View 'filtered_jobs' has columns: {columns}")

    # Create DataFrame from the view
    if data:
        filtered_df = pd.DataFrame(data, columns=columns)
        print(filtered_df.head())  # Optional: print a few rows to verify
    else:
        print("The view 'filtered_jobs' is empty.")
        filtered_df = pd.DataFrame(columns=columns)
except mysql.connector.Error as err:
    print(f"Error querying view: {err}")
    filtered_df = pd.DataFrame()

Error querying view: 1146 (42S02): Table 'au_tech_jobs.filtered_jobs' doesn't exist


In [12]:
# Step 3: Proceed with Further Analysis
if not filtered_df.empty:
    # You can now use filtered_df for building your recommendation system
    print("Data ready for further analysis.")
    rows_before = len(filtered_df)
    print(f"Number of rows before removing duplicates: {rows_before}")
    
else:
    print("No valid data to process.")

No valid data to process.


In [13]:
#handiling dublicates
filtered_df.drop_duplicates(inplace=True)
rows_after = len(filtered_df)
print(f"Number of rows after removing duplicates: {rows_after}")


Number of rows after removing duplicates: 0


In [14]:
# Use TF-IDF and cosine similarity to identify redundant text entries (similar descriptions)
if 'job_description' in filtered_df.columns:
        tfidf = TfidfVectorizer().fit_transform(filtered_df['job_description'].fillna(''))
        similarity_matrix = cosine_similarity(tfidf)

        # Identify pairs with high similarity (> 0.9) and drop redundant ones
        similar_indices = np.where(similarity_matrix > 0.9)
        dropped_indices = set()
        for i, j in zip(*similar_indices):
            if i != j and i < j and j not in dropped_indices:  # Avoid self-comparison and ensure each pair is only handled once
                dropped_indices.add(j)

        filtered_df.drop(index=list(dropped_indices), inplace=True)
        rows_after = len(filtered_df)
        print(f"Number of rows after removing duplicates: {rows_after}")

In [15]:
# Step 5: Handle Missing Values
if 'job_type' in filtered_df.columns and not filtered_df['job_type'].isna().all():
    filtered_df['job_type'] = filtered_df['job_type'].fillna(filtered_df['job_type'].mode()[0])
if 'department' in filtered_df.columns:
    filtered_df['department'] = filtered_df['department'].fillna('Not Specified')
if 'job_description' in filtered_df.columns:
    filtered_df['job_description'] = filtered_df['job_description'].fillna('Not Specified')
if 'summary' in filtered_df.columns:
    filtered_df['summary'] = filtered_df['summary'].fillna('Not Specified')
if 'company_description' in filtered_df.columns:
    filtered_df['company_description'] = filtered_df['company_description'].fillna('Not Specified')

In [16]:
 # Drop rows where critical fields like 'position' are missing
if 'position' in filtered_df.columns:
    filtered_df.dropna(subset=['position'], inplace=True)


In [17]:
# Step 6: Clean HTML Tags from Text Fields
def clean_html(text):
    try:
        return BeautifulSoup(text, "html.parser").get_text() if isinstance(text, str) else text
    except Exception as e:
        print(f"Warning while cleaning HTML: {e}")
        return text

if 'job_description' in filtered_df.columns:
    filtered_df['job_description'] = filtered_df['job_description'].apply(clean_html)
if 'summary' in filtered_df.columns:
    filtered_df['summary'] = filtered_df['summary'].apply(clean_html)
if 'company_description' in filtered_df.columns:
    filtered_df['company_description'] = filtered_df['company_description'].apply(clean_html)

In [18]:
# Step 7: Save the Cleaned Data to an CSV File
filtered_df.to_csv('filtered_df_backup.csv', index=False)
# Step 7: Save the Cleaned Data to an Excel File
filtered_df.to_excel('database_export.xlsx', index=False, engine='openpyxl')

ModuleNotFoundError: No module named 'openpyxl'

In [ ]:
 # Step 8: Content-Based Filtering - Feature Extraction and Similarity Computation
if 'job_description' in filtered_df.columns:
    # Compute TF-IDF matrix for job descriptions
    tfidf_matrix = TfidfVectorizer(stop_words='english', max_features=5000).fit_transform(filtered_df['job_description'])
    # Compute cosine similarity matrix
    cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

    # Function to get job recommendations based on similarity
    def recommend_jobs(job_id, top_n=5):
        if job_id not in filtered_df['job_id'].values:
            print(f"Job ID {job_id} not found in the dataset.")
            return pd.DataFrame()
        idx = filtered_df.index[filtered_df['job_id'] == job_id].tolist()[0]
        similarity_scores = list(enumerate(cosine_sim[idx]))
        similarity_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)
        similar_jobs_indices = [i[0] for i in similarity_scores[1:top_n+1]]
        return filtered_df.iloc[similar_jobs_indices]

    # Example usage
    job_id_example = filtered_df['job_id'].iloc[0]  # Take the first job ID as an example
    recommended_jobs = recommend_jobs(job_id_example)
    print("Recommended Jobs:")
    print(recommended_jobs)
else:
    print("No valid data to process.")


Recommended Jobs:
    job_id                                    job_description  \
9      300  Who we are\n\nLaunching in 2003, Tyro has grow...   
1        2  Who we areLaunching in 2003, Tyro has grown to...   
3        4  Who we areLaunching in 2003, Tyro has grown to...   
4        5  Who we areLaunching in 2003, Tyro has grown to...   
11    1519  Who we areLaunching in 2003, Tyro has grown to...   

                                              summary  \
9                                       Not Specified   
1   'Who we are Launching in 2003, Tyro has grown ...   
3   Who we are Launching in 2003, Tyro has grown t...   
4                                       Not Specified   
11                                      Not Specified   

                            position                 department   job_type  \
9     Desktop Support Engineer (Mac)  Software Engineering - IT  Full-time   
1    Security Consultant - Part-Time  Software Engineering - IT  Part-time   
3             

In [ ]:
# Step 9: Collaborative Filtering - User-Job Interaction Data
# Create a user-job interaction matrix
try:
    cursor.execute("SELECT user_id, job_id FROM applied_job")
    interaction_data = cursor.fetchall()
    interaction_df = pd.DataFrame(interaction_data, columns=['user_id', 'job_id'])
    user_job_matrix = interaction_df.pivot_table(index='user_id', columns='job_id', aggfunc='size', fill_value=0)

    # Use SVD for collaborative filtering
    svd = TruncatedSVD(n_components=10)
    svd_matrix = svd.fit_transform(user_job_matrix)

    # Function to get job recommendations for a user based on collaborative filtering
    def recommend_jobs_for_user(user_id, top_n=5):
        if user_id not in user_job_matrix.index:
            print(f"User ID {user_id} not found in the dataset.")
            return pd.DataFrame()
        user_idx = user_job_matrix.index.get_loc(user_id)
        similarity_scores = cosine_similarity(svd_matrix[user_idx].reshape(1, -1), svd_matrix)[0]
        similar_users_indices = np.argsort(similarity_scores)[::-1]
        recommended_jobs = set()
        for similar_user_idx in similar_users_indices[1:top_n+1]:
            similar_user_id = user_job_matrix.index[similar_user_idx]
            user_jobs = interaction_df[interaction_df['user_id'] == similar_user_id]['job_id'].values
            recommended_jobs.update(user_jobs)
            if len(recommended_jobs) >= top_n:
                break
        return filtered_df[filtered_df['job_id'].isin(recommended_jobs)].head(top_n)

    # Example usage
    user_id_example = interaction_df['user_id'].iloc[0]  # Take the first user ID as an example
    user_recommended_jobs = recommend_jobs_for_user(user_id_example)
    print("User-Based Recommended Jobs:")
    print(user_recommended_jobs)

except mysql.connector.Error as err:
    print(f"Error fetching user-job interaction data: {err}")

else:
    print("No valid data to process.")

User-Based Recommended Jobs:
       job_id                                    job_description  \
305      1735  At SafetyCulture, we help businesses get bette...   
1900      152  Who we are:Airtasker is Australia’s no. 1 mark...   
4172    29763  Software Engineer - Android - Sydney Office - ...   
5155    31271  Founded in 2017, Willow is a global technology...   
12255   41575  Kasada has an exciting opportunity to join our...   

                                                 summary  \
305    'At SafetyCulture, we’re solving problems that...   
1900   Our mission at Airtasker is to empower people ...   
4172   'Software Engineer - Android - Sydney Office -...   
5155   'About us: Founded in 2017, Willow is a global...   
12255  'Kasada is Australia’s Cyber Start-up of the y...   

                               position                 department   job_type  \
305              Staff Android Engineer                    Support  Full-time   
1900          Software Engineer (React)

In [ ]:
def hybrid_recommend_jobs(user_id, job_id, top_n=5, alpha=0.2):
    # Get content-based recommendations
    content_recommendations = recommend_jobs(job_id, top_n=top_n)
    if content_recommendations.empty:
        content_scores = []
    else:
        content_scores = [(row['job_id'], 1.0) for _, row in content_recommendations.iterrows()]

    # Get collaborative filtering recommendations
    collaborative_recommendations = recommend_jobs_for_user(user_id, top_n=top_n)
    if collaborative_recommendations.empty:
        collaborative_scores = []
    else:
        collaborative_scores = [(row['job_id'], 1.0) for _, row in collaborative_recommendations.iterrows()]

    # Combine recommendations with weighted average
    combined_scores = {}
    for job_id, score in content_scores:
        combined_scores[job_id] = combined_scores.get(job_id, 0) + alpha * score
    for job_id, score in collaborative_scores:
        combined_scores[job_id] = combined_scores.get(job_id, 0) + (1 - alpha) * score

    # Sort by combined score and return top_n recommendations
    sorted_jobs = sorted(combined_scores.items(), key=lambda x: x[1], reverse=True)
    recommended_job_ids = [job[0] for job in sorted_jobs[:top_n]]
    return filtered_df[filtered_df['job_id'].isin(recommended_job_ids)]
hybrid_recommendations = hybrid_recommend_jobs(user_id_example, job_id_example)
print("Hybrid Recommended Jobs:")
print(hybrid_recommendations)




Hybrid Recommended Jobs:
       job_id                                    job_description  \
305      1735  At SafetyCulture, we help businesses get bette...   
1900      152  Who we are:Airtasker is Australia’s no. 1 mark...   
4172    29763  Software Engineer - Android - Sydney Office - ...   
5155    31271  Founded in 2017, Willow is a global technology...   
12255   41575  Kasada has an exciting opportunity to join our...   

                                                 summary  \
305    'At SafetyCulture, we’re solving problems that...   
1900   Our mission at Airtasker is to empower people ...   
4172   'Software Engineer - Android - Sydney Office -...   
5155   'About us: Founded in 2017, Willow is a global...   
12255  'Kasada is Australia’s Cyber Start-up of the y...   

                               position                 department   job_type  \
305              Staff Android Engineer                    Support  Full-time   
1900          Software Engineer (React)  So

In [ ]:
 # Step 11: Evaluation Metrics for the Recommendation System
def evaluate_recommendations(true_interactions, predicted_interactions):
    precision = precision_score(true_interactions, predicted_interactions, average='binary')
    recall = recall_score(true_interactions, predicted_interactions, average='binary')
    f1 = f1_score(true_interactions, predicted_interactions, average='binary')
    return precision, recall, f1

# Dummy evaluation example (assuming binary relevance)
true_interactions = [1, 0, 1, 1, 0]  # True labels indicating relevance of the jobs
predicted_interactions = [1, 0, 0, 1, 1]  # Predicted labels from the recommendations
precision, recall, f1 = evaluate_recommendations(true_interactions, predicted_interactions)
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")


Precision: 0.6666666666666666
Recall: 0.6666666666666666
F1 Score: 0.6666666666666666
